<a href="https://colab.research.google.com/github/jddeoro20/Proyecto-Modelos1/blob/main/03_modelo_con_preprocesado_de_datos_estructurados_y_Cat_Boost_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/kaggle'
!chmod 600 /kaggle/kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null


import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
w = pd.read_csv("test.csv")
print ("(rows, columns))", z.shape)


  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.29GB/s]
(rows, columns)) (692500, 21)


In [ ]:
z0 = z[['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'RENDIMIENTO_GLOBAL']]
test_data = w[['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET']]
z0.head()

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,medio-alto
1,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,bajo
2,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,bajo
3,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,alto
4,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,medio-bajo


In [ ]:
def transformar_internet(df):
    # Asegurarse de que la columna sea de tipo cadena antes de llenar valores nulos
    df.loc[:, 'F_TIENEINTERNET'] = df['F_TIENEINTERNET'].fillna('no info')
    internet_map={
        'no info': 0,
        'No': 0,
        'Si': 1
    }
    df.loc[:, 'F_TIENEINTERNET'] = df['F_TIENEINTERNET'].map(internet_map).astype(str)
    return df

def transformar_estrato(df):
    # Asegurarse de que la columna sea de tipo cadena antes de manipular los valores
    df.loc[:, 'F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].astype(str)
    df.loc[:, 'F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].str.extract('(\d+)')
    df.loc[:, 'F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].fillna('0')
    return df

def transformar_matricula(df):
    # Asegurarse de que la columna sea de tipo cadena antes de llenarla con 'no info'
    cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Menos de 500 mil': .250,
        'Entre 500 mil y menos de 1 millón': .75,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Más de 7 millones': 7.75,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'No pagó matrícula': 0,
        'no info': -1}
    df.loc[:, 'E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].fillna('no info')
    df.loc[:, 'E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].map(cmap).astype(str)
    return df

def transformar_horas_trabajo(df):
    # Asegurarse de que la columna sea de tipo cadena antes de rellenarla con '0'
    horas_map = {
        'Más de 30 horas':1,
        '0':0,
        'Menos de 10 horas':1,
        'Entre 11 y 20 horas':1,
        'Entre 21 y 30 horas':1
    }
    df.loc[:, 'E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].fillna('0')
    df.loc[:, 'E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].map(horas_map).astype(str)
    return df

def transformar_rendimiento(df):
    # Convertir los valores de RENDIMIENTO_GLOBAL en números
    y_col = 'RENDIMIENTO_GLOBAL'
    rmap = {'alto': 3, 'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2}

    # Aplicar el mapeo y reemplazar NaN por un valor por defecto (ej. 0)
    df.loc[:, y_col] = df[y_col].map(rmap).fillna(0).astype(int)

    return df

def transformar_datos(train_data, test_data):
    # Aplicar las transformaciones a los conjuntos de datos
    train_data = transformar_internet(train_data)
    train_data = transformar_estrato(train_data)
    train_data = transformar_matricula(train_data)
    train_data = transformar_horas_trabajo(train_data)
    train_data = transformar_rendimiento(train_data)

    test_data = transformar_internet(test_data)
    test_data = transformar_estrato(test_data)
    test_data = transformar_matricula(test_data)
    test_data = transformar_horas_trabajo(test_data)


    return train_data, test_data


# Transformar ambos conjuntos
train_data_transformed, test_data_transformed = transformar_datos(z0, test_data)

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-4056490559.py:15: SyntaxWarning: invalid escape sequence '\d'
  df.loc[:, 'F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].str.extract('(\d+)')


In [ ]:
train_data_transformed

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,6.25,1,3,1,2
1,3.25,0,3,0,0
2,3.25,1,3,1,0
3,4.75,0,4,1,3
4,3.25,1,3,1,1
...,...,...,...,...,...
692495,0.75,1,2,1,2
692496,3.25,1,3,1,0
692497,1.75,1,3,1,1
692498,3.25,1,1,0,0


In [ ]:
test_data_transformed.head()

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET
0,0.25,1,3,1
1,3.25,1,2,1
2,1.75,0,3,1
3,1.75,1,1,0
4,3.25,1,2,1


##Make Cat Boost Classifier


In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [ ]:
Xtr = train_data_transformed.values[:,:-1]
ytr = train_data_transformed["RENDIMIENTO_GLOBAL"].values
print (Xtr.shape, ytr.shape)

(692500, 4) (692500,)


In [ ]:
ytr = ytr.astype(int)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

model = CatBoostClassifier(iterations=1000, # número de iteraciones
                           learning_rate=0.1, # tasa de aprendizaje
                           depth=6, # profundidad del árbol
                           cat_features=[], # columnas categóricas si las hay
                           verbose=200)  # para mostrar información durante el entrenamiento

# Entrenar el modelo con los datos de entrenamiento
model.fit(Xtr, ytr)

# Realizar predicciones sobre el conjunto de prueba (Xts)
preds_tr = model.predict(Xtr)

# Calcular la exactitud
accuracy = accuracy_score(ytr, preds_tr)
print(f'Accuracy: {accuracy:.4f}')


0:	learn: 1.3707287	total: 430ms	remaining: 7m 9s
200:	learn: 1.2890548	total: 1m 16s	remaining: 5m 2s
400:	learn: 1.2889620	total: 2m 33s	remaining: 3m 49s
600:	learn: 1.2889380	total: 4m 5s	remaining: 2m 43s
800:	learn: 1.2889273	total: 5m 21s	remaining: 1m 19s
999:	learn: 1.2889217	total: 6m 35s	remaining: 0us
Accuracy: 0.3765


In [ ]:
Xts = test_data_transformed.values[:,:]
print (Xts.shape)

(296786, 4)


In [ ]:
preds_ts = model.predict(Xts)

In [ ]:
preds_ts = preds_ts.flatten()
print(preds_ts[:50])

[3 1 3 0 1 0 3 3 0 0 2 0 3 3 3 1 3 0 1 1 3 2 2 0 2 3 3 0 0 1 1 1 3 3 1 0 3
 3 0 0 0 2 1 0 0 0 0 2 1 1]


In [ ]:
# Reverse the transformation of the performance
reverse_rmap = {3: 'alto', 0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto'}
preds_ts_reversed = [reverse_rmap[pred] for pred in preds_ts]

# Create a DataFrame for submission
submission_df = pd.DataFrame({'ID': w['ID'], 'RENDIMIENTO_GLOBAL': preds_ts_reversed})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)